In [1]:
%load_ext autoreload
%autoreload 2
import spike_queries

import pandas as pd
import tqdm
import pickle
import random
import itertools

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel, AutoTokenizer, AutoModel, PreTrainedTokenizerFast
import numpy as np
from typing import List
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from typing import Dict, Tuple
from scipy.spatial.distance import cosine as cosine_distance
from collections import defaultdict


2020-12-01 11:04:10.708 INFO    faiss: Loading faiss with AVX2 support.


In [2]:
with open("queries2.txt", "r") as f:
    queries = f.readlines()
    queries = [l.strip() for l in queries]


In [7]:
for i,q in enumerate(queries):
    try:
        assert len(q.split("\t")) == 2
    except:
        print(i, q, len(q.split("\t")))

In [8]:
for i,q in enumerate(queries):
    a = q.count("arg1:") == q.count("arg2:") == 1
    if not a:
        print(i,q)

In [9]:
queries2results = defaultdict(list)
id2query = defaultdict(list)

for id_and_query in queries:
    print(id_and_query)
    query,id = id_and_query.split("\t")
    id2query[int(id)].append(query)
    
print(id2query[1])

<E|U+C>arg1:virus $infection $causes a <E|U+C>arg2:condition	1
<E|U+C>arg1:virus $infection, which $causes a <E|U+C>arg2:condition	1
<E|U+C>arg1:virus $infection $results $in <E|U+C>arg2:condition	1
<E|U+C>arg1:virus $infection, which $results $in <E|U+C>arg2:condition	1
<E|U+C>arg2:condition $is $the $result of <E|U+C>arg1:virus $infection	1
<E|U+C>arg2:condition, which $is $the $result of <E|U+C>arg1:virus $infection	1
<E|U+C>arg2:condition $is $[l=cause|trigger]caused $by <E|U+C>arg1:virus $infection	1
as a result of <E|U+C>arg1:virus $infection, <E|U+C>arg2:condition $can $develop	1
infected <E|U+C>arg1:patients $usually $develop <E|U+C>arg2:[entity]respiratory illness	2
infected <E|U+C>arg1:patients $show $signs $of <E|U+C>arg2:[entity]respiratory illness	2
<E|U+C>arg2:[entity]respiratory illness $is $observed in infected <E|U+C>arg1:patients	2
a <E|U+C>arg1:subset of $patients $progress to <E|U+C>arg2:hemorrhagic fever	3
a <E|U+C>arg1:subset $of $patients $show $signs $of <E|U+C>

In [6]:
# dataset_name = "covid19"
# num_results = 150
# query_type = "syntactic"
# import time

# for i, q_and_id in tqdm.tqdm(enumerate(queries), total = len(queries)):
#     q, id = q_and_id.split("\t")
#     id = int(id)
#     try:
#         df = spike_queries.perform_query(q, dataset_name, num_results, query_type) #previously: word=Hawaii
#         queries2results[id].append(df)
#         time.sleep(2)
#     except Exception as e:
#         print("Error", i+1)
#         print(e)
        
#         continue

100%|██████████| 422/422 [1:00:25<00:00,  8.59s/it]


In [8]:
for id, dfs in queries2results.items():
     for i,df in enumerate(dfs):
         print(id, i)
         queries2results[id][i] = df.dropna(subset=['arg1_first_index',"arg1_last_index","arg2_first_index","arg2_last_index"])

1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
3 0
3 1
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
7 0
7 1
7 2
7 3
7 4
8 0
8 1
8 2
8 3
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
12 0
12 1
12 2
12 3
13 0
13 1
13 2
13 3
13 4
14 0
14 1
14 2
14 3
14 4
14 5
14 6
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
15 11
15 12
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
17 0
17 1
17 2
17 3
18 0
18 1
18 2
18 3
18 4
18 5
19 0
19 1
19 2
19 3
19 4
19 5
19 6
20 0
20 1
20 2
20 3
20 4
21 0
21 1
21 2
21 3
21 4
21 5
22 0
22 1
22 2
22 3
22 4
22 5
23 0
23 1
23 2
23 3
23 4
23 5
23 6
23 7
23 8
23 9
23 10
24 0
24 1
24 2
25 0
25 1
25 2
25 3
26 0
26 1
26 2
27 0
27 1
27 2
28 0
28 1
28 2
29 0
29 1
29 2
29 3
29 4
29 5
29 6
30 0
30 1
30 2
30 3
30 4
31 0
31 1
31 2
31 3
31 4
31 5
31 6
32 0
32 1
32 2
32 3
32 4
32 5
32 6
33 0
33 1
33 2
33 3
34 0
34 1
34 2
35 

In [11]:
for id, dfs in queries2results.items():
      for i,df in enumerate(dfs):
            queries2results[id][i]["spike_query"] = id2query[id][i]

In [33]:
with open("queries2results2.pickle", "wb") as f:
      pickle.dump(queries2results, f)

In [2]:
with open("queries2results2.pickle", "rb") as f:
    queries2results = pickle.load(f)

In [7]:
queries2results[1][0]

,sentence_id,arg1,arg2,title,article_link,arg1_first_index,arg1_last_index,arg2_first_index,arg2_last_index,sentence_text,paragraph_text,spike_query
0,11709,The virus infection,generalized infections,Speaker Presentations,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,0,2,4,5,The virus infection causes generalized infecti...,The co-emergence of multiple CA-MRSA lineages ...,<E|U+C>arg1:virus $infection $causes a <E|U+C>...
1,16550,Middle East respiratory syndrome coronavirus (...,an acute respiratory illness,"Clinicopathologic, Immunohistochemical, and Ul...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,0,8,10,13,Middle East respiratory syndrome coronavirus (...,Middle East respiratory syndrome coronavirus (...,<E|U+C>arg1:virus $infection $causes a <E|U+C>...
2,42944,MHV infection,changes,viruses Of Mice and Men: The Coronavirus MHV a...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,0,1,17,17,MHV infection increases iron uptake due to tis...,"MHV causes hepatitis, which results in elevate...",<E|U+C>arg1:virus $infection $causes a <E|U+C>...
3,51830,ZIKV infection,mild and self-limiting disease,ARTICLE IN PRESS,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,3,4,7,10,"7 , 8 ZIKV infection usually causes mild and s...",The Flavivirus genus comprises many viruses di...,<E|U+C>arg1:virus $infection $causes a <E|U+C>...
4,54089,SARS-CoV-2 infection,a relatively low release of pro-inflammatory c...,viruses Of Mice and Men: The Coronavirus MHV a...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,14,15,25,33,A possible reason for this is the suppression ...,SARS-CoV-2 replicates more efficiently than SA...,<E|U+C>arg1:virus $infection $causes a <E|U+C>...
...,...,...,...,...,...,...,...,...,...,...,...,...
145,2477435,Fetal cytomegalovirus infection,neurologic damage,Editorial Addressing the Effects of Establishe...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,0,2,4,5,Fetal cytomegalovirus infection causes neurolo...,Also in this issue is an update by Davis et al...,<E|U+C>arg1:virus $infection $causes a <E|U+C>...
146,2497840,SARS-CoV infection,severe symptoms,Pathogenesis of SARS 1145,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,10,11,14,15,"1 , 3 In contrast with previously described co...",A novel coronavirus was identified as the etio...,<E|U+C>arg1:virus $infection $causes a <E|U+C>...
147,2525863,ZIKV infection,"placental pathology , fetal growth restriction...",Animal models of congenital zika syndrome prov...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,0,1,8,17,ZIKV infection during the first trimester also...,"Sheep are a less recognized model of CZS, whic...",<E|U+C>arg1:virus $infection $causes a <E|U+C>...
148,2539632,ZIKV infection,reduced oxygen permeability of the placental v...,Animal models of congenital zika syndrome prov...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,4,5,12,18,"In rhesus macaques , ZIKV infection ( Cambodia...",Rhesus macaques are the most widely utilized N...,<E|U+C>arg1:virus $infection $causes a <E|U+C>...


In [13]:
# queries2results_new = dict()
# for k,dfs in queries2results.items():
#     if k == 0: continue
#     queries2results_new[k] = pd.concat(dfs, axis=0)
    
# queries2results = queries2results_new

In [14]:
ids = set([int(l.split("\t")[-1]) for l in queries])

In [15]:
def add_arguments(sent:str, arg1_start, arg1_end, arg2_start, arg2_end):
    
        s_lst = sent.split(" ")
        if arg1_start > arg2_start:
            arg1_start, arg2_start = arg2_start, arg1_start
            arg1_end, arg2_end = arg2_end, arg1_end
            arg1_str, arg2_str = "{{ARG2:", "<<ARG1:"
        else:
            arg1_str, arg2_str = "<<ARG1:", "{{ARG2:"
        
        s_with_args = s_lst[:arg1_start] + [arg1_str] + s_lst[arg1_start:arg1_end+1] + [">>"] + s_lst[arg1_end+1:arg2_start] + [arg2_str] + s_lst[arg2_start:arg2_end+1] + ["}}"] +s_lst[arg2_end+1:]  
        #s_with_args = s_lst[:arg1_start] + [arg1_str+s_lst[arg1_ind]] + s_lst[arg1_ind+1:arg2_ind] + [arg2_str+s_lst[arg2_ind]] + s_lst[arg2_ind+1:]
        s_with_args = " ".join(s_with_args).replace("ARG1: ", "ARG1:").replace("ARG2: ", "ARG2:")
        s_with_args = s_with_args.replace(" >>", ">>").replace(" }}", "}}")
        return s_with_args
    

In [16]:
NUM_PAIRS_PER_RELATION = 1400

data = []

for id in tqdm.tqdm(ids): #foreach relation
    
    pattern_pairs = list(itertools.product(range(len(id2query[id])), repeat=2))
    pattern_pairs_different = [(p1,p2) for (p1,p2) in pattern_pairs if p1 != p2]
    pattern_pairs_same = [(p1,p2) for (p1,p2) in pattern_pairs if p1 == p2]
    pattern_pairs_different += pattern_pairs_same
    sum_pattern = 0
    relation_examples = []
    
    for p1, p2 in pattern_pairs_different:
        df1,df2 = queries2results[id][p1], queries2results[id][p2]
        if df1.empty or df2.empty: continue
            
        sentences1, sentences2 = df1["sentence_text"].tolist(), df2["sentence_text"].tolist()
        query1, query2 = df1["spike_query"].tolist()[0], df2["spike_query"].tolist()[0]
        arg1_first_start, arg1_first_end = df1["arg1_first_index"].tolist(), df1["arg1_last_index"].tolist()
        arg2_first_start, arg2_first_end = df1["arg2_first_index"].tolist(), df1["arg2_last_index"].tolist()
        arg1_second_start, arg1_second_end = df2["arg1_first_index"].tolist(), df2["arg1_last_index"].tolist()
        arg2_second_start, arg2_second_end = df2["arg2_first_index"].tolist(), df2["arg2_last_index"].tolist()
        
        
        all_pair_combinations = list(itertools.product(range(len(df1)), range(len(df2))))
        random.shuffle(all_pair_combinations)
        for combination in all_pair_combinations[:100]:
            ind1, ind2 = combination
            
            sent1, sent2 = sentences1[ind1], sentences2[ind2]
            sent1_arg1_start, sent1_arg1_end = arg1_first_start[ind1], arg1_first_end[ind1]
            sent1_arg2_start, sent1_arg2_end = arg2_first_start[ind1], arg2_first_end[ind1]
            sent2_arg1_start, sent2_arg1_end = arg1_second_start[ind2], arg1_second_end[ind2]
            sent2_arg2_start, sent2_arg2_end = arg2_second_start[ind2], arg2_second_end[ind2]
            sent1_with_args = add_arguments(sent1, sent1_arg1_start, sent1_arg1_end, sent1_arg2_start, sent1_arg2_end)
            sent2_with_args = add_arguments(sent2, sent2_arg1_start, sent2_arg1_end, sent2_arg2_start, sent2_arg2_end)
            
            sent1_lst = sent1.split(" ")
            sent2_lst = sent2.split(" ")
            
            sent1_arg1_w = sent1_lst[sent1_arg1_start:sent1_arg1_end+1]
            sent1_arg2_w = sent1_lst[sent1_arg2_start:sent1_arg2_end+1]
            sent2_arg1_w = sent2_lst[sent2_arg1_start:sent2_arg1_end+1]
            sent2_arg2_w = sent2_lst[sent2_arg2_start:sent2_arg2_end+1]
            
            d = {"first": sent1_with_args, "second": sent2, "second_with_arguments": sent2_with_args, "query_first": query1, "query_second": query2, 
                 "first_arg1": (sent1_arg1_start, sent1_arg1_end), "first_arg2": (sent1_arg2_start, sent1_arg2_end), 
                 "second_arg1": (sent2_arg1_start, sent2_arg1_end),
                 "second_arg2": (sent2_arg2_start, sent2_arg2_end), "first_arg1_words": sent1_arg2_w,
                "first_arg2_words": sent1_arg2_w, "second_arg1_words": sent2_arg1_w, "second_arg2_words": sent2_arg2_w,
                "query_id": id}
            relation_examples.append(d)
    
    random.shuffle(relation_examples)
    relation_examples = relation_examples[:NUM_PAIRS_PER_RELATION]
    data.extend(relation_examples)
    
random.shuffle(data)

100%|██████████| 61/61 [00:18<00:00,  3.34it/s]


In [17]:
data[0]

{'first': 'O. pedunculosum contains <<ARG1:3-O-methylquercetin glucosides>> that inhibit lipopolysaccharideinduced NO production in {{ARG2:macrophages}} ( Wan et al. 2012 ) , and a mannan-specific lectin with antifungal activity ( He et al. 2011 ) .',
 'second': 'However , they can do even better than that : Prof. Clemen Posten and his team at the Karlsruhe Institute of Technology are working on bioprocesses that enable them to cultivate the green alga Chlamydomonas reinhardtii in a way that triggers it to produce environmentallyfriendly hydrogen , both more cheaply and more energy efficiently than has been possible up until now .',
 'second_with_arguments': 'However , they can do even better than that : Prof. Clemen Posten and his team at the Karlsruhe Institute of Technology are working on bioprocesses that enable them to cultivate the green alga Chlamydomonas reinhardtii in <<ARG1:a way>> that triggers it to produce environmentallyfriendly {{ARG2:hydrogen}} , both more cheaply and m

*** Add negative examples ***

In [28]:
lengths = [len(d["first"].split(" ")) + len(d["second"].split(" ")) for d in data]
data = [d for i,d in enumerate(data) if lengths[i] < 250]
data = [d for d in data if (d["second_arg1"] != d["second_arg2"]) and (d["first_arg1"] != d["first_arg2"])]
data = [d for d in data if d["first_arg1_words"] != [''] and d["first_arg2_words"] != ['']
       and d["second_arg1_words"] != [''] and d["second_arg2_words"] != ['']]
data = [d for d in data if '' not in d["first_arg1_words"] and '' not in d["first_arg2_words"]
       and '' not in d["second_arg1_words"] and '' not in d["second_arg2_words"]]

def is_number_between_brackets(sent, ind):
    lst = sent.split(" ")
    if not is_number(lst[ind]): return False
    if ind == len(lst) - 1 or ind == 0: return False
    return (lst[ind -1] == "[" and lst[ind + 1] == "]") or (lst[ind -1] == "(" and lst[ind + 1] == ")")

def is_number(s):
    try:
        d = int(s)
        return True
    except:
        return False

data = [d for d in data if not is_number_between_brackets(d["second"], d["second_arg1"][0]) and not
       is_number_between_brackets(d["second"], d["second_arg2"][0]) and not
        is_number_between_brackets(d["first"], d["first_arg1"][0]) and not
        is_number_between_brackets(d["first"], d["first_arg2"][0])]

np.random.seed(0)
ids_subset = np.random.choice(list(ids), size = int(0.2 * len(ids)), replace = False)
data_train = [d for d in data if d["query_id"] not in ids_subset]
data_dev = [d for d in data if d["query_id"] in ids_subset]
print(ids_subset)

[27 36 61 29 12  3 35 60 41 23  5 11]


In [29]:
same = [d for d in data if d["query_first"] == d["query_second"]]
not_same = [d for d in data if d["query_first"] != d["query_second"]]
print(len(same), len(not_same))

11594 55515


In [32]:
d = data_train[330]
d

{'first': 'RESULTS : Acidosis limited the cellular consumption of glucose and ATP , causing tumor cells to <<ARG1:enter a metabolically dormant but energetically economic state>> , which promoted tumor cell survival during {{ARG2:glucose deficiency}} .',
 'second': 'How might clonal hematopoiesis in aged populations increase their risk of death from COVID-19 infection ?',
 'second_with_arguments': 'How might <<ARG1:clonal hematopoiesis>> in aged populations increase their risk of death from {{ARG2:COVID-19 infection}} ?',
 'query_first': '<E|U+C>arg1:something, that improved $survival in <E|U+C>arg2:[e=DISEASE|PATHOLOGICAL_FORMATION|CANCER]something',
 'query_second': '<E|U+C>arg1:something can $[w=decrease|prevent|increase]decrease $death rates in <E|U+C>arg2:[e=DISEASE|CANCER|PATHOLOGICAL_FORMATION]something',
 'first_arg1': (16, 23),
 'first_arg2': (31, 32),
 'second_arg1': (2, 3),
 'second_arg2': (13, 14),
 'first_arg1_words': ['glucose', 'deficiency'],
 'first_arg2_words': ['gluco

In [31]:
not_same[20]

{'first': '<<ARG1:Cerebral venous sinus thrombosis>> is a rare complication of {{ARG2:retrosigmoid resection of CPA tumor}} .',
 'second': 'Other complications caused by cataracts are lens subluxation or luxation and glaucoma .',
 'second_with_arguments': 'Other complications caused by {{ARG2:cataracts>> are <<ARG1:lens subluxation or luxation and glaucoma}} .',
 'query_first': '<E|U+C>arg1:[e]stroke is a $complication of <E|U+C>arg2:flu :vaccination',
 'query_second': 'among the $complications trigerred $by <E|U+C>arg2:[e=DISEASE|PATHOLOGICAL_FORMATION]asthma is <E|U+C>arg1:[e]stroke',
 'first_arg1': (0, 3),
 'first_arg2': (9, 13),
 'second_arg1': (6, 11),
 'second_arg2': (4, 4),
 'first_arg1_words': ['retrosigmoid', 'resection', 'of', 'CPA', 'tumor'],
 'first_arg2_words': ['retrosigmoid', 'resection', 'of', 'CPA', 'tumor'],
 'second_arg1_words': ['lens',
  'subluxation',
  'or',
  'luxation',
  'and',
  'glaucoma'],
 'second_arg2_words': ['cataracts'],
 'query_id': 7}

In [33]:
with open("data.pickle", "wb") as f:
    pickle.dump(data, f)
    
with open("data_dev.pickle", "wb") as f:
    pickle.dump(data_dev, f)
    
with open("data_train.pickle", "wb") as f:
    pickle.dump(data_train, f)
    
import json

def write(data, fname):
    
    with open(fname, "w") as f:
        for d in data:
            first, second = d["first"], d["second_with_arguments"]
            first_arg1 = d["first_arg1"]
            first_arg2 = d["first_arg2"]
            second_arg1 = d["second_arg1"]
            second_arg2 = d["second_arg2"]
        
            elems = [first, second, first_arg1, first_arg2, second_arg1, second_arg2]
            keys = ["first", "second", "first_arg1", "first_arg2"]
        
            f.write(json.dumps(d) + "\n")
        
            #f.write(d["first"] + "\t" + d["second"] + "\t" + d["query"] + "\t" + "-".join(d["first_arg1"])+ "\n")
            
    
write(data, "data.txt")
write(data_dev, "data_dev.txt")
write(data_train, "data_train.txt")

In [34]:
len(data_train), len(data_dev)

(55459, 11650)

In [29]:
for i in range(20):
    print(data[i]["first"], data[i]["first_arg1"], data[i]["first_arg2"])
    print(data[i]["second_with_arguments"])
    print("---------------")
    print(data[i]["query_first"])
    print(data[i]["query_second"])
    print("============================")

<<ARG1:Old World monkeys ( OWM , referring to Africa and Asia ) and hominoid primates>> {{ARG2:hominoid primates}} ( apes , humans ) , share a common ancestor . (0, 14) (13, 14)
This indicates that the tremoviruses , <<ARG1:hepatovirus and lineage>> {{ARG2:lineage 1}} viruses have a common ancestor and evolved in their natural hosts , independently .
---------------
<E|U+C>arg1:something $and <E|U+C>arg2:something share $a $common $[w=origin|ancestry|ancestor]ancestry
<E|U+C>arg1:something $and <E|U+C>arg2:something share $a $common $[w=origin|ancestry|ancestor]ancestry
The zoonosis caused by <<ARG1:Dirofilaria immitis>> , a parasite of {{ARG2:dogs and other mammals}} , is transmitted by mosquitos and black flies to humans . (4, 5) (10, 13)
N. gonorrhoeae infects columnar cells ; <<ARG1:it>> is therefore a parasite of {{ARG2:the cervix}} , not the vagina .
---------------
<E|U+C>arg1:something, $a <E|U+C>arg2:[e]bats $parasite
<E|U+C>arg1:something $is $a <E|U+C>arg2:[e=ORGANISM|ORGAN|

In [25]:
print(len(data))

67045


In [31]:
lengths = [len(d["first"].split(" ")) + len(d["second"].split(" ")) for d in data]

In [32]:
np.mean(lengths), np.std(lengths), np.max(lengths), np.min(lengths)

(67.92299198429576, 22.057857454680867, 237, 16)

In [58]:
len([l for l in lengths if l > 250])

0